In [2]:
import pandas as pd
import sqlite3
import sqlalchemy
import random
from datetime import timedelta
import numpy as np

In [3]:
reactions = pd.read_csv('sampleReactions.csv', sep="\t")
hashtags = ["#stayhome", "#lockdown", "#lockdown", "#socialdistancing", "#socialdistancing", "#lockdown", "#longcovid", "#freeagain", "#rip", "#unemployed", "#homeschooling", "#homeoffice", "#rip", "#unemployed", "#homeschooling", "#homeoffice", "#workfromhome", "#workfromhome", "#stayhome", "#stayhome", "#vaccination", "#quarantine", "#flattenthecurve", "#stayhome", "#lockdown", "#longcovid","","","","","","","","","","","","","","","",""]
for (i, row) in enumerate(reactions.iterrows()):
    reactions.loc[i, 'comment'] = row[1]['comment'] + " " +random.choice(hashtags)
    
reactions['comment']


0      while most the country was all about Baby Shar...
1                         What happened with Chernobyl? 
2      Surprised there wasn’t another Baby Shark spik...
3      Montana being the only state with Far Cry 5 wa...
4      Fortnite lasted longer than George Floydd, BLM...
                             ...                        
470    I remember these huge ships getting paid to ta...
471    Why does the simulation end up making the patc...
472    Notice the steady stream coming out of China. ...
473    this seems like bullshit... &#x200B; In Januar...
474    What’s brilliant about that is it works like a...
Name: comment, Length: 475, dtype: object

In [5]:
df = pd.read_json("./sampleComments.json")
df['id'] = df.index
df = df.drop(['value'], axis=1)
df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
df['date'] = df['date'].dt.date
df['createdAt'] = pd.to_datetime('today').normalize()
df['updatedAt'] = pd.to_datetime('today').normalize()

#df.drop(df[df.country != "Germany"].index, inplace=True)

for (i, row) in enumerate(df.iterrows()):
    reaction = reactions.sample()
    
    df.loc[i, 'exactDate'] = random.choice([True, False])
    df.loc[i, 'flagged'] =  random.random() < .05
    df.loc[i, 'comment'] = reaction['comment'].values[0]
    #df.loc[i, 'author'] = reaction['author'].values[0]
    #if (i % 3) is 0:
        #newDate = row[1].date + timedelta(days=random.randint(5,20))
        #df.loc[i, 'enddate'] = newDate

df

,date,comment,weight,country,id,createdAt,updatedAt,exactDate,flagged
0,2021-01-22,Fun fact: Rian Johnson directed the lowest rat...,4,Fiji,0,2021-03-24,2021-03-24,False,False
1,2020-06-11,"Well, it was a nice planet while we lasted. Ho...",4,Lebanon,1,2021-03-24,2021-03-24,True,False
2,2020-08-10,gotta make sure those administrators can eat s...,4,Iceland,2,2021-03-24,2021-03-24,True,False
3,2020-07-26,This is a prime example of data cherry picking...,4,Lesotho,3,2021-03-24,2021-03-24,False,False
4,2020-09-08,This is why anyone who thought the market was ...,3,Mongolia,4,2021-03-24,2021-03-24,False,False
...,...,...,...,...,...,...,...,...,...
49995,2020-11-05,"“Person of Interest,” AKA, “Yeah, pretty consi...",3,El Salvador,49995,2021-03-24,2021-03-24,False,False
49996,2020-03-16,Tuition really started soaring when student lo...,4,Latvia,49996,2021-03-24,2021-03-24,True,False
49997,2020-06-29,I read this in the Simpsons Comic Book Guy voice,4,Iceland,49997,2021-03-24,2021-03-24,True,False
49998,2020-10-31,College is more of a business than an actual e...,4,Luxembourg,49998,2021-03-24,2021-03-24,True,False


In [13]:
sqlite = sqlite3.connect("../server/coronaMemories.sqlite")
#drop and recreate to make sure primary keys are in the db
create = """CREATE TABLE memories (id INTEGER PRIMARY KEY,
                       comment TEXT NOT NULL,
                       country NOT NULL,
                       date DATE NOT NULL,
                       weight INTEGER,
                       exactDate INTEGER DEFAULT 0,
                       flagged INTEGER DEFAULT 0,
                       createdAt TIMESTAMP,
                       updatedAT TIMESTAMP);"""

with sqlite as con:
    con.execute('DROP TABLE memories;')
    print(con.execute(create))
    
df.to_sql(con=sqlite, index=False, name='memories', if_exists="append")